In [1]:
import pandas as pd
import pickle

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from pandas import json_normalize

from sklearn.preprocessing import StandardScaler

from random import randint

In [2]:
with open("model.pkl", "rb") as f:
    kmeans = pickle.load(f)

In [3]:
kmeans

KMeans(n_clusters=5)

In [4]:
top100_songs_df = pd.read_csv("top100_songs.csv")
top100_songs_df

,title,artists
0,Fast Car,Luke Combs
1,Last Night,Morgan Wallen
2,Need A Favor,Jelly Roll
3,Take Two,BTS
4,Save Me (with Lainey Wilson),Jelly Roll
...,...,...
95,Memory Lane,Old Dominion
96,A Little Bit Happy,TALK
97,Perfect,Ed Sheeran
98,Whiskey Glasses,Morgan Wallen


In [5]:
clustered_songs_df = pd.read_csv("Clustered_songs.csv")
clustered_songs_df

,Title,Artist,Cluster
0,You're The One - Un mal pour un bien,Petula Clark,0
1,Doctor My Eyes,Jackson Browne,4
2,Falling in Love at a Coffee Shop,Landon Pigg,4
3,Peaceful Easy Feeling - 2013 Remaster,Eagles,4
4,Summertime Blues,Eddie Cochran,0
...,...,...,...
10117,Elements (Deluxe),Ludovico Einaudi,1
10118,Moon Safari,Air,4
10119,Islands - Essential Einaudi,Ludovico Einaudi,1
10120,Liquid Mind II: Slow World,Liquid Mind,2


# Spotify Authentication

In [6]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secret_dict = {}

for line in string.split("\n"):
    if len(line) > 1:
        
        secret_dict[line.split(':')[0]]=line.split(':')[1].strip()
    

In [7]:
auth_manager = SpotifyClientCredentials(client_id = secret_dict['clientid'], 
                                        client_secret = secret_dict['clientsecret'])

sp = spotipy.Spotify(auth_manager = auth_manager)

In [52]:
result = sp.search(q = "kesariya", type = "track")
result

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=kesariya&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1wRPtKGflJrBx9BmLsSwlU'},
       'href': 'https://api.spotify.com/v1/artists/1wRPtKGflJrBx9BmLsSwlU',
       'id': '1wRPtKGflJrBx9BmLsSwlU',
       'name': 'Pritam',
       'type': 'artist',
       'uri': 'spotify:artist:1wRPtKGflJrBx9BmLsSwlU'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/2fMqTqiTxUDlmcOEPaQSsx'},
       'href': 'https://api.spotify.com/v1/artists/2fMqTqiTxUDlmcOEPaQSsx',
       'id': '2fMqTqiTxUDlmcOEPaQSsx',
       'name': 'Amitabh Bhattacharya',
       'type': 'artist',
       'uri': 'spotify:artist:2fMqTqiTxUDlmcOEPaQSsx'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      

In [55]:
result["tracks"]["items"][0]["uri"]

len(result["tracks"]["items"])

10

In [10]:
af = sp.audio_features("spotify:track:59O4w7NZgVqowQkcG8gs1i")
af

[{'danceability': 0.866,
  'energy': 0.525,
  'key': 7,
  'loudness': -9.391,
  'mode': 1,
  'speechiness': 0.104,
  'acousticness': 0.00348,
  'instrumentalness': 0.00996,
  'liveness': 0.0698,
  'valence': 0.452,
  'tempo': 140.019,
  'type': 'audio_features',
  'id': '59O4w7NZgVqowQkcG8gs1i',
  'uri': 'spotify:track:59O4w7NZgVqowQkcG8gs1i',
  'track_href': 'https://api.spotify.com/v1/tracks/59O4w7NZgVqowQkcG8gs1i',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/59O4w7NZgVqowQkcG8gs1i',
  'duration_ms': 233169,
  'time_signature': 4}]

In [11]:


df = pd.json_normalize(af[0])
df = df.drop(["type", "id", "uri", "track_href", "analysis_url", "time_signature"], axis = 1)
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.866,0.525,7,-9.391,1,0.104,0.00348,0.00996,0.0698,0.452,140.019,233169


In [12]:
X_scaled = StandardScaler().fit_transform(df)
X_scaled
cluster = kmeans.predict(df)
cluster[0]

4

In [13]:
df = clustered_songs_df[clustered_songs_df.Cluster == 4].reset_index(inplace= False)
rand_int = randint(0, len(df))
print(rand_int)
song = df.iloc[[rand_int]]
# print(type(song))
s_list = [song.Title, song.Artist]
# print(s_list)
# song["Title"]
# print(str(song.Artist))

print(df.Title[rand_int])
df.Artist[rand_int]

960
Leaving on a Jet Plane


'Peter, Paul and Mary'

# Functions for recommendations and predictions

In [56]:
# Pick random songs from the predicted cluster.
def pick_song(cluster):
    
    df = clustered_songs_df[clustered_songs_df.Cluster == cluster].reset_index()
    rand_int = randint(0, len(df))
    return (df.Title[rand_int], df.Artist[rand_int])
    

In [57]:
# Predict cluster of input song
def predict_cluster(audio_features):
    
    df = pd.json_normalize(af[0])
    df = df.drop(["type", "id", "uri", "track_href", "analysis_url", "time_signature"], axis = 1)
    cluster = kmeans.predict(df)[0]
    return pick_song(cluster)


In [58]:
# Get audio features of the input song
def get_audio_features(input_song):
    
    results = sp.search(q = input_song, type = "track")
    
    if len(results["tracks"]["items"]) > 1: 
        
        track_id = result["tracks"]["items"][0]["uri"]
        audio_features = sp.audio_features(track_id)
        return predict_cluster(audio_features)
    else:
        return "No recommendation. Please retry later"    

In [59]:
import random

# Recommend from hot songs
def random_songs(data):
    
    random_number = random.randint(0, len(data) - 1)
    return (data.title[random_number], data.artists[random_number])

# Function to check if hot songs or not
def check_and_recommend(song_name, top100_songs_df):
    
    songs = list(map(str.lower, top100_songs_df.title))
    
    if song_name.lower() in songs:
        
        recommended_song = random_songs(top100_songs_df)
        return "Its hot song!! Check this out as well {0} by {1} ".format(recommended_song[0], recommended_song[1])
    
    else:
        
        recommended_song = get_audio_features(song_name)
        
        if type(recommended_song) == str:
            
            return recommended_song
        else:
            
            st = "Hey do check this song as well,  {0} by {1}".format(recommended_song[0], recommended_song[1])
            return st
    

In [34]:
user_input = input("Type a song title: ")

recommendation = check_and_recommend(user_input, top100_songs_df)
print(recommendation)

Type a song title: take two
Its hot song!! Check this out as well Makeba by Jain 


In [35]:
user_input = input("Type a song title: ")

recommendation = check_and_recommend(user_input, top100_songs_df)
print(recommendation)

Type a song title: Doctor My Eyes
Hey do check this song as well,  Peanut Butter Conspiracy by Jimmy Buffett


In [49]:
user_input = input("Type a song title: ")

recommendation = check_and_recommend(user_input, top100_songs_df)
print(recommendation)

Type a song title: sdjfnsdkjfn
No recommendation. Please retry later


In [60]:
user_input = input("Type a song title: ")

recommendation = check_and_recommend(user_input, top100_songs_df)
print(recommendation)

Type a song title: kesariya
Hey do check this song as well,  Rockaria! by Electric Light Orchestra
